Neural networks work best on normalized data, and the raw data is not normal.  This notebook calculates and contains summary statistics to normalize topo, fuel, and wind data.

In [20]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymongo as pm
import pyprind
import collections
import matplotlib as mpl
import pickle
import datetime as dt
import geopandas as gpd
from collections import defaultdict

cmap = mpl.colors.ListedColormap(['black', 'red', 'yellow', 'blue', 'green'])

%matplotlib inline

In [21]:
FBFM40 = pd.read_csv('FBFM40convert.csv', header=0, index_col=0)

fbfm40simple = dict()
for row in FBFM40.itertuples():
    fbfm40simple[row[0]]   = sum(row[2:6])

fbfm = np.zeros([205])

for item in fbfm40simple:
    fbfm[item] = fbfm40simple[item]

def fbfm40func(a):
    try:
        return fbfm[a]
    except:
        return 0

vfunc = np.vectorize(fbfm40func)

test = np.array([[104, 113],
                [-5, 9999]])

In [22]:
vfunc(test)

array([[2.15, 0.  ],
       [0.  , 0.  ]])

In [23]:
def convolute(raster, coords):
    """
    Takes in a raster and a coordinate list of [left, right, top, bottom]
    Returns convolution"""
    return raster[coords[2]:coords[3],coords[0]:coords[1]]

def convolute3d(raster, coords):
    """
    Takes in a raster and a coordinate list of [left, right, top, bottom, t0, t1]
    Returns convolution"""
    return raster[coords[4]:coords[5],coords[2]:coords[3],coords[0]:coords[1]]

def coord_gen(shape, conv_size, stride):
    """
    Takes in a np.shape, and desired convolution size and stride.
    Returns a valid list of coordinates for that shape
    """
    coords = []
    xs = np.arange(0, shape[1]-conv_size, stride)
    ys = np.arange(0, shape[0]-conv_size, stride)
    for y in ys:
        for x in xs:
            new_coord = [x, x+conv_size, y, y+conv_size]
            coords.append(new_coord)
    return(coords)

def perimeter_finder(ar, br, conv_size, stride):
    """
    ar and br are two fire perimeters from the same fire separated by a reasonable interval
    conv_size and stride and the convolution size and stride
    """
    
    interesting_coords = []

    for coord in coord_gen(ar.shape, conv_size, stride):
        ac = convolute(ar, coord)
        bc = convolute(br, coord)
        max_area = ac.shape[0]*ac.shape[1]
        if 0 < np.sum(ac) and np.sum(ac) < max_area:
            interesting_coords.append(coord)
    return interesting_coords

def perimeter_finder2(ar, br, conv_size, stride):
    """
    ar and br are two fire perimeters from the same fire separated by a reasonable interval
    conv_size and stride and the convolution size and stride
    """
    
    interesting_coords = []

    for coord in coord_gen(ar.shape, conv_size, stride):
        ac = convolute(ar, coord)
        bc = convolute(br, coord)
        max_area = ac.shape[0]*ac.shape[1]
        if 256*256*0.2 < np.sum(ac) and np.sum(ac) < max_area:
            interesting_coords.append(coord)
    return interesting_coords

def perimeter_finder3(ar, br, conv_size, stride):
    """
    ar and br are two fire perimeters from the same fire separated by a reasonable interval
    conv_size and stride and the convolution size and stride
    """
    
    interesting_coords = []

    for coord in coord_gen(ar.shape, conv_size, stride):
        ac = convolute(ar, coord)
        bc = convolute(br, coord)
        max_area = ac.shape[0]*ac.shape[1]
        if 256*256*0.2 < np.sum(ac) and np.sum(ac) < max_area*0.8:
            interesting_coords.append(coord)
    return interesting_coords

def azimuthx(a):
    return np.cos(np.radians(a))

def azimuthy(a):
    return np.sin(np.radians(a))

def bytes_to_raster(buf, shape):
    br = np.frombuffer(buf, dtype=np.int8)
    r = br.reshape(shape)
    return r

def db_update(ic):
    c_perim1 = convolute(perim1['raster'], ic)
    c_perim0 = convolute(perim0['raster'], ic)
    #c_dem = convolute(topo['us_dem'], ic)
    c_slp = convolute(topo['us_slp'], ic)  
    c_asp = convolute(topo['us_asp'], ic)  
    c_fbfm40 = convolute(topo['us_130fbfm40'], ic)  

    c=np.zeros([5, 256, 256])

    c[0,:,:] = c_perim1
    c[1,:,:] = c_perim0
    c[2,:,:] = c_asp
    c[3,:,:] = c_slp
    c[4,:,:] = vfunc(c_fbfm40)
    
    c = c.astype(np.int8)


    topo_shape = perim1['raster'].shape

    enddate = dt.datetime.strptime(atmo['enddate'], '%Y-%m-%d')
    startdate = dt.datetime.strptime(atmo['startdate'], '%Y-%m-%d')

    zero = startdate + dt.timedelta(hours=-1000)
    now = dt.datetime.strptime(perim0['date'], '%Y-%m-%d')
    predict = dt.datetime.strptime(perim1['date'], '%Y-%m-%d')

    t1 = enddate-predict

    t1_index = t1.days*24

    t0 = enddate-now

    t0_index = t0.days*24

    atmo_shape = atmo['windspeed'].shape[1:]
    conversion_factor = np.mean(topo_shape[0]/atmo_shape[0]+topo_shape[1]/atmo_shape[1])
    atmo_coords = np.round(ic/conversion_factor, 0)
    atmo_coords = [int(x) for x in atmo_coords]
    atmo_coords = [-t0_index, -t1_index] + atmo_coords
    ac =atmo_coords
    if ac[1] == 0:
        ac[1] = -1
        ac[0] = ac[0] -1

    c_winddir   = atmo['winddirection'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
    c_windspeed = atmo['windspeed'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
    c_precip    = atmo['precipitation'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
    c_humid     = atmo['humidity'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
    c_temp      = atmo['temperature'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]

    a = np.zeros([5, t0_index-t1_index, c_winddir.shape[1], c_winddir.shape[2]])

    a[0, :, :, :] = c_winddir
    a[1, :, :, :] = c_windspeed
    a[2  :, :, :] = c_precip
    a[3, :, :, :] = c_humid
    a[4, :, :, :] = c_temp -273

    return data

In [24]:
atmo_files=os.listdir('wind')
topo_files=os.listdir('topo')
perim_files = os.listdir('perims/clean')

fire_ids =  [x.split(' ')[0] for x in atmo_files]
print(len(fire_ids))
#fire_ids.remove('ID-SCF-G4A0')
#fire_ids.remove('WA-OWF-G70S')
fire_ids.remove('AK-GADC-GW3W')
fire_ids.remove('AK-YTDC-GY4W')

print(len(fire_ids))

139
137


In [25]:
#if you want to run this quickly, 32 stride
num_conv = 642893 #perimeterfinder1, 32 stride
num_conv2 = 322467 #perimeterfinder2, 32 stride
num_conv3 = 247223 #perimeterfinder3, 32 stride
num_conv4 = 61782 #perimeterfinder4, 64 stride
num_conv5 = 61500 #perimeterfinder4, 64 stride, with big ID and WA fires

In [89]:
errors = []
test = []

slp_max = 0
slp_min = 0
asp_max = 0
asp_min = 0
fuel_max = 0
fuel_min = 0

winddir_max = 0
winddir_min = 0
windspeed_max = 0
windspeed_min = 0
precip_max = 0
precip_min = 0
humid_max = 0
humid_min = 0
temp_max = 0
temp_min = 0

progbar = pyprind.ProgBar(num_conv5, width=80, update_interval=5)

for fire_id in fire_ids[0:20]:

    op = open('wind/'+fire_id+' wind.pkl', 'rb')
    atmo = pickle.load(op)
    op.close()

    op = open('topo/'+fire_id+' topo.pkl', 'rb')
    topo = pickle.load(op)
    op.close()

    layers = ['us_slp', 'us_dem', 'us_asp', 'us_130fbfm40']

    for layer in layers:
        topo[layer] = topo[layer].T

    perims = [x for x in perim_files if fire_id in x]
    perims.sort()

    for s in range(len(perims)-1):
        op = open('perims/clean/'+perims[s], 'rb')
        perim0 = pickle.load(op)
        op.close()

        op = open('perims/clean/'+perims[s+1], 'rb')
        perim1 = pickle.load(op)
        op.close()

        perim0['raster'] = perim0['raster'].astype(int)
        perim1['raster'] = perim1['raster'].astype(int)
        #print(perim1['date'], perim0['date'])

        ics = perimeter_finder3(perim0['raster'], perim1['raster'], 256, 64)


        
        for ic in ics:
            c_slp = convolute(topo['us_slp'], ic)  
            c_asp = convolute(topo['us_asp'], ic)  
            c_fbfm40 = convolute(topo['us_130fbfm40'], ic)  

            slp_max = np.max([slp_max, np.max(c_slp)])
            slp_min = np.min([slp_min, np.min(c_slp)])
            asp_max = np.max([asp_max, np.max(c_asp)])
            asp_min = np.min([asp_min, np.min(c_asp)])
            fuel_max = np.max([fuel_max, np.max(vfunc(c_fbfm40))])
            fuel_min = np.min([fuel_min, np.min(vfunc(c_fbfm40))])

            topo_shape = perim1['raster'].shape

            enddate = dt.datetime.strptime(atmo['enddate'], '%Y-%m-%d')
            startdate = dt.datetime.strptime(atmo['startdate'], '%Y-%m-%d')

            zero = startdate + dt.timedelta(hours=-1000)
            now = dt.datetime.strptime(perim0['date'], '%Y-%m-%d')
            predict = dt.datetime.strptime(perim1['date'], '%Y-%m-%d')

            t1 = enddate-predict

            t1_index = t1.days*24

            t0 = enddate-now

            t0_index = t0.days*24

            atmo_shape = atmo['windspeed'].shape[1:]
            conversion_factor = np.mean(topo_shape[0]/atmo_shape[0]+topo_shape[1]/atmo_shape[1])
            atmo_coords = np.round(ic/conversion_factor, 0)
            atmo_coords = [int(x) for x in atmo_coords]
            atmo_coords = [-t0_index, -t1_index] + atmo_coords
            ac =atmo_coords
            if ac[1] == 0:
                ac[1] = -1
                ac[0] = ac[0] -1

            c_winddir   = atmo['winddirection'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
            c_windspeed = atmo['windspeed'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
            c_precip    = atmo['precipitation'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
            c_humid     = atmo['humidity'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
            c_temp      = atmo['temperature'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]

            a = np.zeros([5, t0_index-t1_index, c_winddir.shape[1], c_winddir.shape[2]])

            winddir_max = np.max([winddir_max,np.max(c_winddir)])
            winddir_min = np.min([winddir_min, np.min(c_winddir)])
            windspeed_max = np.max([windspeed_max, np.max(c_windspeed)])
            windspeed_min = np.min([windspeed_min, np.min(c_windspeed)])
            precip_max = np.max([precip_max, np.max(c_precip)])
            precip_min = np.min([precip_min, np.min(c_precip)])
            humid_max = np.max([humid_max, np.max(c_humid)])
            humid_min = np.min([humid_min, np.min(c_humid)])
            temp_max = np.max([temp_max, np.max(c_temp)])
            temp_min = np.min([temp_max, np.min(c_temp)])
            progbar.update()

0% [##########                                                                      ] 100% | ETA: 00:26:37

In [90]:
topo_bounds = {'slp_max': slp_max,
              'slp_min': slp_min,
              'asp_max': asp_max,
              'asp_min': asp_min,
              'fuel_max': fuel_max,
              'fuel_min': fuel_min}
topo_bounds

{'slp_max': 69,
 'slp_min': 0,
 'asp_max': 359,
 'asp_min': -1,
 'fuel_max': 14,
 'fuel_min': 0}

In [115]:
atmo_bounds = {'winddir_max': winddir_max,
              'winddir_min': winddir_min,
              'windspeed_max': windspeed_max,
              'windspeed_min': windspeed_min,
              'precip_max': precip_max,
              'precip_min': precip_min,
              'humid_max':humid_max,
              'humid_min': humid_min,
              'temp_max': temp_max,
              'temp_min': temp_min}
atmo_bounds

{'winddir_max': 359.9945068359375,
 'winddir_min': 0.0,
 'windspeed_max': 20.490127563476562,
 'windspeed_min': 0.0,
 'precip_max': 404.3092041015625,
 'precip_min': 0.0,
 'humid_max': 996.71923828125,
 'humid_min': 0.0,
 'temp_max': 311.3910217285156,
 'temp_min': 290.467529296875}

In [92]:
{'slp_max': 69,
 'slp_min': 0,
 'asp_max': 359,
 'asp_min': -1,
 'fuel_max': 14,
 'fuel_min': 0}

{'winddir_max': 359.9945068359375,
 'winddir_min': 0.0,
 'windspeed_max': 20.490127563476562,
 'windspeed_min': 0.0,
 'precip_max': 404.3092041015625,
 'precip_min': 0.0,
 'humid_max': 996.71923828125,
 'humid_min': 0.0,
 'temp_max': 311.3910217285156,
 'temp_min': 290.467529296875}

{'winddir_max': 359.9945068359375,
 'winddir_min': 0.0,
 'windspeed_max': 20.490127563476562,
 'windspeed_min': 0.0,
 'precip_max': 404.3092041015625,
 'precip_min': 0.0,
 'temp_max': 311.3910217285156,
 'temp_min': 290.467529296875}

In [106]:
from collections import defaultdict

In [31]:
def get_dists(ic):
    global slp_dict
    global asp_dict
    global fuel_dict

    global winddir_array
    global windspeed_array
    global precip_array
    global humid_array
    global temp_array
    
    
#     c_slp = convolute(topo['us_slp'], ic)  
#     c_asp = convolute(topo['us_asp'], ic)  
#     c_fbfm40 = convolute(topo['us_130fbfm40'], ic)

#     for a in c_slp.ravel():
#         slp_dict[a] += 1
#     for a in c_asp.ravel():
#         asp_dict[a] += 1
#     for a in c_fbfm40.ravel():
#         fuel_dict[a] += 1

    topo_shape = perim1['raster'].shape

    enddate = dt.datetime.strptime(atmo['enddate'], '%Y-%m-%d')
    startdate = dt.datetime.strptime(atmo['startdate'], '%Y-%m-%d')

    zero = startdate + dt.timedelta(hours=-1000)
    now = dt.datetime.strptime(perim0['date'], '%Y-%m-%d')
    predict = dt.datetime.strptime(perim1['date'], '%Y-%m-%d')

    t1 = enddate-predict

    t1_index = t1.days*24

    t0 = enddate-now

    t0_index = t0.days*24

    atmo_shape = atmo['windspeed'].shape[1:]
    conversion_factor = np.mean(topo_shape[0]/atmo_shape[0]+topo_shape[1]/atmo_shape[1])
    atmo_coords = np.round(ic/conversion_factor, 0)
    atmo_coords = [int(x) for x in atmo_coords]
    atmo_coords = [-t0_index, -t1_index] + atmo_coords
    ac =atmo_coords
    if ac[1] == 0:
        ac[1] = -1
        ac[0] = ac[0] -1

#     c_winddir   = atmo['winddirection'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
#     c_windspeed = atmo['windspeed'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
    c_precip    = atmo['precipitation'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
    c_humid     = atmo['humidity'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]
    c_temp      = atmo['temperature'][ac[0]:ac[1], ac[2]:ac[3], ac[4]:ac[5]]

#     winddir_array = np.append(winddir_array, c_winddir.ravel())
#     windspeed_array = np.append(windspeed_array, c_windspeed.ravel())
    precip_array = np.append(precip_array, c_precip.ravel())
    humid_array = np.append(humid_array, c_humid.ravel())
    temp_array = np.append(c_temp, c_temp.ravel())

In [32]:
import multiprocessing as multi

In [33]:
progbar=pyprind.ProgBar(num_conv5, width=80, update_interval=5)

# slp_dict = defaultdict(int)
# asp_dict = defaultdict(int)
# fuel_dict = defaultdict(int)

# winddir_array = np.array(0)
# windspeed_array = np.array(0)
precip_array = np.array(0)
humid_array = np.array(0)
temp_array = np.array(300)

for fire_id in fire_ids:

    op = open('wind/'+fire_id+' wind.pkl', 'rb')
    atmo = pickle.load(op)
    op.close()

    op = open('topo/'+fire_id+' topo.pkl', 'rb')
    topo = pickle.load(op)
    op.close()

    layers = ['us_slp', 'us_dem', 'us_asp', 'us_130fbfm40']

    for layer in layers:
        topo[layer] = topo[layer].T

    perims = [x for x in perim_files if fire_id in x]
    perims.sort()

    for s in range(len(perims)-1):
        op = open('perims/clean/'+perims[s], 'rb')
        perim0 = pickle.load(op)
        op.close()

        op = open('perims/clean/'+perims[s+1], 'rb')
        perim1 = pickle.load(op)
        op.close()

        perim0['raster'] = perim0['raster'].astype(int)
        perim1['raster'] = perim1['raster'].astype(int)
        #print(perim1['date'], perim0['date'])

        ics = perimeter_finder3(perim0['raster'], perim1['raster'], 256, 64)

        for ic in ics:
            get_dists(ic)
            progbar.update()

0% [############################################################################### ] 100% | ETA: 00:00:01
Total time elapsed: 01:24:22


KeyboardInterrupt: 

In [34]:
precip_mu =  np.mean(precip_array)
humid_mu = np.mean(humid_array)
temp_mu = np.mean(temp_array)

precip_std = np.std(precip_array)
humid_std =  np.std(humid_array)
temp_std = np.std(temp_array)



In [146]:
winddir_mu = np.mean(winddir_array)
windspeed_mu = np.mean(windspeed_array)
precip_mu =  np.mean(precip_array)
humid_mu = np.mean(humid_array)
temp_mu = np.mean(temp_array)

winddir_std = np.std(winddir_array)
windspeed_std = np.std(windspeed_array)
precip_std = np.std(precip_array)
humid_std =  np.std(humid_array)
temp_std = np.std(temp_array)

mu_and_stds= {'winddir_mu' :  winddir_mu,
              'windspeed_mu': windspeed_mu,
              'precip_mu': precip_mu,
              'humid_mu': humid_mu,
              'temp_mu' : temp_mu,
              'winddir_std' : winddir_std,
              'windspeed_std': windspeed_std,
              'precip_std': precip_std,
              'humid_std': humid_std,
              'temp_std': temp_std}

In [147]:
mu_and_stds

{'winddir_mu': 197.44300700576673,
 'windspeed_mu': 3.471466034172764,
 'precip_mu': 0.3038395194508763,
 'humid_mu': 0.3038395194508763,
 'temp_mu': 146.45264,
 'winddir_std': 103.5507390863788,
 'windspeed_std': 2.4904719136862505,
 'precip_std': 5.285449460470769,
 'humid_std': 5.285449460470769,
 'temp_std': 146.48682}

In [8]:
#mu_and_std
{'winddir_mu': 197.44300700576673,
 'windspeed_mu': 3.471466034172764,
 'precip_mu': 0.3038395194508763,
 'humid_mu': 0.3038395194508763,
 'temp_mu': 146.45264,
 'winddir_std': 103.5507390863788,
 'windspeed_std': 2.4904719136862505,
 'precip_std': 5.285449460470769,
 'humid_std': 5.285449460470769,
 'temp_std': 146.48682}

#slp_dict

slp_dict =            {0: 534297233,
             1: 521435708,
             2: 267375937,
             3: 204156335,
             4: 159813773,
             5: 147622120,
             6: 131141844,
             7: 120773511,
             8: 114087917,
             9: 112754020,
             10: 108044843,
             11: 108379264,
             12: 103165318,
             13: 102350973,
             14: 101118755,
             15: 99185555,
             16: 98933216,
             17: 99186186,
             19: 95240047,
             18: 98040342,
             21: 92521646,
             20: 94573084,
             23: 89372087,
             24: 89086059,
             22: 91839161,
             28: 79982769,
             30: 71575965,
             27: 81771674,
             29: 74332570,
             26: 85253111,
             32: 59637923,
             31: 66899712,
             33: 52292074,
             25: 87119677,
             34: 45666466,
             36: 30552155,
             35: 38484029,
             37: 25048876,
             39: 14030161,
             38: 18820542,
             41: 7590656,
             40: 10295264,
             42: 5510128,
             43: 4025160,
             44: 2942641,
             46: 1587051,
             45: 2182887,
             47: 1212728,
             49: 693820,
             48: 894307,
             51: 440262,
             52: 340671,
             50: 553038,
             55: 179087,
             57: 114411,
             56: 155146,
             53: 282355,
             54: 224529,
             58: 95273,
             59: 78228,
             60: 64516,
             64: 18733,
             65: 13862,
             62: 37676,
             63: 27158,
             61: 46922,
             66: 11167,
             68: 9276,
             67: 9368,
             69: 4704,
             70: 3309,
             71: 2229,
             74: 466,
             72: 1208,
             75: 74,
             73: 956}
    
#asp dict

asp_dict =             {251: 12881863,
             266: 11506007,
             270: 17894651,
             282: 11668869,
             265: 10147768,
             274: 10231891,
             285: 10443881,
             288: 12805292,
             302: 10559609,
             261: 11068730,
             257: 11611469,
             296: 11511241,
             295: 10252049,
             236: 10818946,
             291: 10651089,
             281: 10671459,
             284: 11653847,
             278: 11013343,
             244: 10430658,
             229: 9961009,
             237: 10916830,
             303: 10516918,
             299: 9599014,
             262: 9643374,
             263: 9434082,
             273: 11380256,
             280: 10184466,
             219: 9872623,
             231: 9867350,
             315: 18506012,
             243: 11878971,
             277: 9876246,
             293: 9983563,
             292: 9327714,
             276: 9451429,
             327: 9656610,
             286: 9903836,
             275: 9548173,
             264: 9386448,
             267: 9400192,
             272: 9352764,
             290: 9318941,
             254: 10347295,
             289: 9332763,
             283: 7981769,
             241: 9169861,
             249: 9195374,
             248: 10683808,
             256: 7679669,
             279: 8691844,
             246: 10092953,
             255: 11582793,
             233: 9925031,
             258: 10474919,
             253: 9773401,
             247: 9275334,
             260: 8536120,
             271: 9044515,
             268: 9041937,
             298: 9158584,
             225: 20740278,
             230: 10058919,
             259: 10097568,
             240: 9736207,
             217: 8603808,
             200: 8430762,
             308: 9417892,
             309: 9669229,
             198: 11414501,
             252: 8525748,
             250: 9080940,
             305: 8263890,
             300: 10765155,
             320: 9305380,
             269: 4572478,
             238: 7096636,
             321: 9139654,
             209: 9293338,
             287: 8770144,
             242: 9075865,
             226: 7495011,
             297: 9029161,
             294: 8249275,
             245: 8127367,
             239: 11145307,
             301: 7096918,
             204: 7557970,
             202: 8613200,
             212: 10457647,
             326: 9538429,
             325: 8754108,
             232: 8724690,
             192: 10348887,
             196: 8823062,
             207: 8690056,
             330: 8511473,
             312: 8089294,
             306: 9444340,
             222: 8490343,
             310: 9402243,
             304: 9469446,
             206: 10707588,
             199: 8413998,
             201: 9754018,
             205: 9704806,
             227: 8393017,
             333: 10028351,
             331: 8155895,
             318: 8816373,
             210: 10619820,
             215: 8288482,
             345: 9289504,
             355: 8039240,
             350: 6805700,
             329: 9548406,
             197: 7846660,
             189: 7733960,
             188: 9834558,
             323: 8919088,
             334: 8868975,
             3: 9196164,
             9: 6797801,
             346: 6317990,
             195: 9239380,
             187: 8691899,
             185: 8360603,
             186: 8502274,
             167: 10204739,
             338: 8935799,
             311: 9018783,
             347: 9330415,
             8: 8825629,
             18: 10909330,
             20: 7696311,
             357: 7331612,
             235: 9718396,
             221: 9530781,
             190: 9034979,
             163: 8878406,
             158: 9647622,
             341: 10575675,
             332: 7889640,
             319: 9222375,
             317: 7942939,
             307: 8402831,
             356: 9183177,
             353: 7464897,
             220: 9915019,
             27: 7822585,
             75: 11023095,
             113: 10293882,
             141: 9850993,
             162: 8015819,
             170: 7724517,
             349: 8059247,
             343: 7931692,
             351: 8730807,
             13: 6297578,
             47: 7900425,
             83: 9145078,
             98: 10907262,
             128: 10085324,
             339: 7694589,
             324: 7772267,
             337: 7843832,
             336: 8474813,
             218: 9789942,
             0: 14186466,
             25: 8940071,
             59: 10316434,
             86: 11182049,
             108: 12609402,
             135: 19310318,
             328: 6515185,
             16: 8055336,
             50: 9578023,
             65: 7702987,
             76: 7419370,
             79: 9776536,
             78: 10128442,
             63: 10880388,
             6: 7457972,
             11: 8570427,
             38: 9122405,
             57: 10409258,
             56: 10238971,
             35: 7509692,
             335: 7106978,
             348: 8394833,
             64: 9824532,
             87: 9169600,
             62: 8532106,
             61: 8650759,
             72: 8128985,
             67: 8771415,
             164: 9162782,
             234: 8470262,
             12: 9462729,
             32: 9910877,
             344: 8419560,
             340: 7573184,
             22: 7785387,
             91: 8990571,
             88: 9023592,
             70: 8758635,
             60: 9162738,
             71: 12181659,
             68: 10052868,
             194: 10334866,
             15: 8505528,
             73: 9234327,
             54: 7912930,
             45: 19574664,
             26: 10265884,
             180: 15266222,
             149: 10547820,
             107: 9008786,
             94: 10086173,
             81: 10583319,
             14: 9437545,
             41: 8679723,
             101: 10520657,
             122: 11151787,
             130: 10109747,
             131: 9785656,
             116: 11588564,
             96: 9332852,
             213: 10432479,
             1: 7098800,
             90: 17289924,
             109: 9633185,
             118: 9678562,
             117: 9532827,
             110: 9480409,
             342: 7128889,
             7: 7747335,
             31: 6209412,
             181: 8233314,
             23: 8500443,
             19: 7533063,
             55: 9192142,
             80: 8281013,
             100: 10176490,
             115: 10752269,
             104: 11466055,
             42: 7673238,
             37: 7882749,
             52: 8233634,
             208: 8676484,
             228: 9508934,
             21: 8938284,
             102: 11624599,
             354: 7427112,
             313: 7436853,
             322: 7961105,
             39: 9343862,
             92: 9214666,
             97: 9755573,
             358: 7117525,
             352: 7737041,
             17: 7009495,
             28: 8092295,
             48: 8976048,
             69: 8641069,
             4: 8138205,
             10: 8108449,
             5: 7506970,
             24: 6931659,
             33: 9707918,
             40: 9247902,
             29: 8547717,
             316: 7316749,
             36: 9001859,
             82: 9387575,
             34: 8801182,
             93: 11222183,
             95: 9399107,
             89: 4484215,
             84: 9163586,
             99: 8667441,
             114: 8628205,
             106: 9978952,
             51: 9303142,
             44: 2709311,
             119: 10122889,
             120: 11230484,
             2: 7282395,
             112: 9707613,
             123: 10813194,
             74: 9824915,
             105: 10469655,
             124: 10066910,
             58: 6641937,
             126: 10130242,
             359: 3634783,
             30: 9642860,
             49: 9394174,
             43: 7013298,
             53: 9316897,
             66: 9371185,
             77: 11115112,
             103: 8050797,
             150: 9361211,
             46: 7014941,
             127: 9129559,
             138: 9656534,
             139: 9861091,
             133: 8106338,
             147: 10456871,
             143: 9694242,
             145: 9644345,
             111: 10776782,
             121: 7656106,
             136: 8122033,
             211: 6820874,
             191: 9430018,
             168: 9281624,
             216: 9786603,
             184: 9026342,
             151: 8858451,
             174: 8388237,
             85: 9964844,
             173: 8430513,
             183: 10086158,
             169: 9046188,
             176: 10009608,
             155: 7900091,
             175: 9056007,
             142: 8896849,
             223: 7472517,
             214: 9583199,
             193: 7080100,
             178: 8187204,
             177: 8301494,
             203: 9308691,
             171: 9763846,
             165: 10151053,
             314: 3115732,
             161: 10998316,
             179: 4042301,
             182: 8293664,
             160: 8639835,
             166: 7079812,
             153: 10319379,
             137: 8659760,
             125: 8903191,
             154: 9695225,
             172: 8635276,
             157: 8778449,
             224: 2858676,
             132: 8896215,
             140: 9776820,
             129: 10075222,
             148: 7182770,
             134: 3213815,
             146: 10095817,
             144: 8486925,
             156: 9371127,
             159: 8486447,
             152: 8726286}

#fuel dict. note these are grid values and not loads

fuel_dict_raw =            {122: 737543630,
             121: 324121353,
             142: 143427081,
             141: 18014529,
             145: 82503726,
             102: 1393888610,
             101: 260787496,
             93: 67464993,
             91: 28368020,
             161: 128449042,
             147: 95552718,
             183: 274488376,
             165: 415824218,
             99: 169798127,
             162: 9368124,
             186: 96884917,
             188: 144533535,
             182: 22295415,
             185: 36264031,
             98: 55882094,
             104: 2845998,
             143: 1744398,
             92: 3371509,
             103: 6027455,
             184: 59963153,
             181: 19326643,
             146: 210815,
             163: 3849936,
             187: 35368247,
             189: 4448572,
             144: 716521,
             201: 123866,
             107: 2619747,
             106: 116163,
             149: 706050,
             123: 793210,
             105: 6658393,
             108: 47159,
             148: 67150,
             202: 1146884}

In [14]:
fuel_dict = defaultdict(int)

for item in fuel_dict_raw:
    index =  fbfm[item]
    fuel_dict[index] += fuel_dict_raw[item]

In [18]:
counter = 0
total = 0

for item in fuel_dict:
    total += fuel_dict[item]*item
    counter += fuel_dict[item]
    
fuel_mu = total/counter

rolling = 0
for item in fuel_dict:
    rolling += fuel_dict[item]*(item-fuel_mu)**2

fuel_std = rolling/total

fuel_mu, fuel_std

(3.1791731372052965, 3.849030795578499)

In [19]:
counter = 0
total = 0

for item in slp_dict:
    total += slp_dict[item]*item
    counter += slp_dict[item]
    
slp_mu = total/counter

rolling = 0
for item in slp_dict:
    rolling += slp_dict[item]*(item-slp_mu)**2

slp_std = rolling/total

(slp_mu, slp_std)

(12.456316426670947, 10.252085180944922)

In [36]:
precip_mu, precip_std, humid_mu, humid_std, temp_mu, temp_std

(0.3351187873020621,
 5.54332198942828,
 54.31366149257656,
 97.00557445908757,
 288.4088,
 2.2353404)

In [37]:
#final array, note that we clip humidity to be 100

{'winddir_mu': 197.44300700576673,
 'windspeed_mu': 3.471466034172764,
 'precip_mu': 0.3351187873020621,
 'humid_mu': 70.86907293192847,
 'temp_mu': 288.4088,
 'winddir_std': 103.5507390863788,
 'windspeed_std': 2.4904719136862505,
 'precip_std': 5.54332198942828,
 'humid_std': 32.79107440957776,
 'temp_std': 2.2353404,
 'slp_mu': 12.456316426670947,
 'slp_std': 10.252085180944922,
 'fuel_mu': 3.1791731372052965,
 'fuel_std': 3.849030795578499}

{'winddir_mu': 197.44300700576673,
 'windspeed_mu': 3.471466034172764,
 'precip_mu': 0.3351187873020621,
 'humid_mu': 54.31366149257656,
 'temp_mu': 288.4088,
 'winddir_std': 103.5507390863788,
 'windspeed_std': 2.4904719136862505,
 'precip_std': 5.54332198942828,
 'humid_std': 97.00557445908757,
 'temp_std': 2.2353404,
 'slp_mu': 12.456316426670947,
 'slp_std': 10.252085180944922,
 'fuel_mu': 3.1791731372052965,
 'fuel_std': 3.849030795578499}

In [42]:
np.sum(humid_array> 100)

332255

In [43]:
len(humid_array)

14216521

In [50]:
humid_array2 = np.append(humid_array[humid_array < 100], [100]*len(humid_array > 100))

In [51]:
np.mean(humid_array2)

70.86907293192847

In [52]:
np.std(humid_array2)

32.79107440957776

0.0